In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets trl
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) 
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [4]:
# Model from Hugging Face hub
base_model = "openai-community/gpt2"

# New instruction dataset
dataset_id = "AbhishekJosyula/Wikisun_QA_Format"

# Fine-tuned model
new_model = "gpt-wikisum-ft"

In [6]:
dataset = load_dataset(dataset_id, split='train[:10%]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/180889 [00:00<?, ? examples/s]

In [7]:
def format_ds(sample):
  sample['Text'].replace("[INST]", "<s>[INST]")
  sample['Text'] += "</s>"
  return sample

dataset = dataset.map(format_ds)

Map:   0%|          | 0/18089 [00:00<?, ? examples/s]

In [8]:
dataset = dataset.train_test_split(test_size=0.1)

In [9]:
dataset = dataset.with_format("torch")

In [10]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [11]:
    # "google/flan-t5-base",
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [13]:
peft_params = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
)

In [14]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=1000,
    logging_steps=500,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [15]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    peft_config=peft_params,
    dataset_text_field="Text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/16280 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:437: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
25,3.068500
50,2.692800
75,2.913300
100,2.692000
125,2.827000
150,2.481900
175,2.748900
200,2.499400
225,2.887300
250,2.318700


TrainOutput(global_step=4070, training_loss=2.485262481352035, metrics={'train_runtime': 3087.8881, 'train_samples_per_second': 5.272, 'train_steps_per_second': 1.318, 'total_flos': 5682476407246848.0, 'train_loss': 2.485262481352035, 'epoch': 1.0})

In [16]:
trainer.model.save_pretrained(new_model, push_to_hub=True)
trainer.tokenizer.save_pretrained(new_model, push_to_hub=True)

('gpt-wikisum-ft/tokenizer_config.json',
 'gpt-wikisum-ft/special_tokens_map.json',
 'gpt-wikisum-ft/vocab.json',
 'gpt-wikisum-ft/merges.txt',
 'gpt-wikisum-ft/added_tokens.json',
 'gpt-wikisum-ft/tokenizer.json')

In [17]:
testprompt = dataset['test'][0]['Text'].split("[/INST]")

In [18]:
testprompt[1]

' Andy Kunasek is an American politician from Arizona. A member of the Republican Party, Hickman served on the Maricopa County Board of Supervisors representing the 3rd district from 1997 until 2017. Kunasek was first appointed to the county board in 1997 to replace Betsey Bayless, when she became secretary of state. == Career == Kunasek supports selling Chase Field, home of the Arizona Diamondbacks. In a letter to team owner Ken Kendrick, Kunasek wrote “take your stupid baseball team and get out” and go back to “f–king West Virginia,”\n\nKunasek received a $123,110 settlement from Maricopa County after being targeted by former Sheriff Joe Arpaio and former County Attorney Andrew Thomas.\n\nKunasek was a noted champion of the county park system, including naming a peak in the Estrella Mountains in honor of fallon Navy Seal Charles Keating IV.1 </s>'

In [19]:
prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<s>[INST] Who is Leonardo Da Vinci? [/INST] Leonardo Da Vinci – founder of the Italian motorcycle and motorcyclist car brands, who was born in Bologna, Italy and lives in Rome. He was the first man to own the company Dacia, but later changed his name to Ducati in his own name, and began driving on his own account only after becoming the owner of the Italian magazine Sinoi Pic.  [/INST] Who is Leonardo Da Vinci? == (C.D. 2000-2005) Da Vinci is an independent motorcycle enthusiast with an interest in the motorcycle industry and the family business, and an avid photographer.

He was born in Bologna in Rome, Italy. His parents are Italian-American, and his parents were born in the United States. He attended the University of Otinoparino in 1989—the son of Marco Dampino and a student. == Dacia == With a wife


In [20]:
print(trainer.model.base_model)

LoraModel(
  (model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): lora.Linear4bit(
              (base_layer): Linear4bit(in_features=768, out_features=2304, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=768, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=2304, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
            )
            (c_proj): Linear4bit(in_fea

In [21]:
trainer.push_to_hub("AbhishekJosyula/gpt-wikisum-ft")

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1708195746.7772fafd3cbc.586.0:   0%|          | 0.00/31.0k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AbhishekJosyula/results/commit/e81883940c9b2c4a263606e60b20d01629f71bc8', commit_message='AbhishekJosyula/gpt-wikisum-ft', commit_description='', oid='e81883940c9b2c4a263606e60b20d01629f71bc8', pr_url=None, pr_revision=None, pr_num=None)

In [31]:
testprompt = dataset['test'][7]['Text'].split("[/INST]")
prompt = testprompt[0].split("[INST]")[1]
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<s>[INST]  Patrick Bjorkstrand (born July 1, 1992) is a Danish professional ice hockey forward who is currently an unrestricted free agent. He most recently played within the Ontario Reign of the AHL while under contract to the Los Angeles Kings of the National Hockey League (NHL).

He is the son of coach Todd Bjorkstrand and the older brother of Oliver Bjorkstrand.   [/INST]  Patrick Bjorkstrand (born July 1, 1992) is a Danish professional ice hockey forward who is currently an unrestricted free agent. He most recently played within the Ontario Reign of the AHL while under contract to the Los Angeles Kings of the National Hockey League (NHL).

He is the son of coach Todd Bjorkstrand and the older sister of Oliver Bjorkstrand. == Career Statistics == After he was acquired from the Vancouver Canucks in exchange for defenseman Mark Scheifele, Bjorkstrand played for Denmark's national


In [32]:
result[0]['generated_text'].split("[/INST]")

['<s>[INST]  Patrick Bjorkstrand (born July 1, 1992) is a Danish professional ice hockey forward who is currently an unrestricted free agent. He most recently played within the Ontario Reign of the AHL while under contract to the Los Angeles Kings of the National Hockey League (NHL).\n\nHe is the son of coach Todd Bjorkstrand and the older brother of Oliver Bjorkstrand.   ',
 "  Patrick Bjorkstrand (born July 1, 1992) is a Danish professional ice hockey forward who is currently an unrestricted free agent. He most recently played within the Ontario Reign of the AHL while under contract to the Los Angeles Kings of the National Hockey League (NHL).\n\nHe is the son of coach Todd Bjorkstrand and the older sister of Oliver Bjorkstrand. == Career Statistics == After he was acquired from the Vancouver Canucks in exchange for defenseman Mark Scheifele, Bjorkstrand played for Denmark's national"]

In [33]:
prompt2="Patrick Bjorkstrand (born July 1, 1992) is a Danish professional ice hockey forward who is currently an unrestricted free agent. He most recently played within the Ontario Reign of the AHL while under contract to the Los Angeles Kings of the National Hockey League (NHL).\n\nHe is the son of coach Todd Bjorkstrand and the older sister of Oliver Bjorkstrand. == Career Statistics == After he was acquired from the Vancouver Canucks in exchange for defenseman Mark Scheifele, Bjorkstrand played for Denmark's national"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt2} [/INST]")
print(result[0]['generated_text'].split("[/INST]"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


["<s>[INST] Patrick Bjorkstrand (born July 1, 1992) is a Danish professional ice hockey forward who is currently an unrestricted free agent. He most recently played within the Ontario Reign of the AHL while under contract to the Los Angeles Kings of the National Hockey League (NHL).\n\nHe is the son of coach Todd Bjorkstrand and the older sister of Oliver Bjorkstrand. == Career Statistics == After he was acquired from the Vancouver Canucks in exchange for defenseman Mark Scheifele, Bjorkstrand played for Denmark's national ", ' Patrick Bjorkstrand (born July 1, 1992) is a Danish professional hockey player whose pro career was cut short during the 2004-2005 season after the Canucks drafted him after he had returned from Boston, and played for the Red Sea-area New Jersey Devils in the 2014-2015 campaign. He was drafted by the Ottawa Senators in the fourth round (No. 9) in the 2007-08 NHL Entry']


In [34]:
result[0]['generated_text'].split("[/INST]")

["<s>[INST] Patrick Bjorkstrand (born July 1, 1992) is a Danish professional ice hockey forward who is currently an unrestricted free agent. He most recently played within the Ontario Reign of the AHL while under contract to the Los Angeles Kings of the National Hockey League (NHL).\n\nHe is the son of coach Todd Bjorkstrand and the older sister of Oliver Bjorkstrand. == Career Statistics == After he was acquired from the Vancouver Canucks in exchange for defenseman Mark Scheifele, Bjorkstrand played for Denmark's national ",
 ' Patrick Bjorkstrand (born July 1, 1992) is a Danish professional hockey player whose pro career was cut short during the 2004-2005 season after the Canucks drafted him after he had returned from Boston, and played for the Red Sea-area New Jersey Devils in the 2014-2015 campaign. He was drafted by the Ottawa Senators in the fourth round (No. 9) in the 2007-08 NHL Entry']